In [160]:
from bs4 import BeautifulSoup
import requests
import time
import re
import os
import pandas as pd
import mysql.connector

# Saves all item URLs to the file "deal.txt"

In [168]:
deals_url = ''

for k in range(1, 11):
    URL = 'https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?_pgn='+ str(k)
    headers = {'User-agent': 'Mozilla/5.0'} 
    
    response = requests.get(URL + "&_pgn=" + str(k), headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    link = soup.find_all('a', class_ = "s-item__link")
    
    for i in range(0, len(link)):
        item_url = link[i].get('href')
        deals_url = deals_url + '\n' + item_url
    
with open('deals.txt', 'w') as file:
    file.write(deals_url.lstrip())

# Opens the file above and downloads each of the pages (URLs) into the folders "deals"

In [169]:
path = 'deals'
os.mkdir(path)

In [170]:
read_deals = open('deals.txt', 'r').read()
deals_url_split = read_deals.split('\n')


for item in range(0, len(deals_url_split)):
    URL = deals_url_split[item]
    response = requests.get(URL, headers = headers)
    epid_id = re.sub(r'.+/(\[0-9]{12})\?.*', r'\1', URL)
    
    with open(os.path.join(os.getcwd() + '/deals', str(epid_id) + '.htm'), 'w') as file:
        file.write(response.text)
    
    time.sleep(4)

# Select Information

Write a separate piece of code that loops through the pages you downloaded in the "deals" folder and opens and parses them into a Python xxxxsoup-object. 

Identify and select: seller name, seller score, item price, item price, list price, number of items sold, title, returns allowed, shipping price, and condition

In [299]:
pages = os.listdir('./deals')
deals_df = pd.DataFrame(columns = ['seller_names', 'seller_scores', 'item_prices', 'list_prices', 'list_prices_yes', 'num_items_sold', 'title', 'returns', 'shipping_price', 'conditions'])
i = 0

for page in pages:
    file = open(os.path.join(os.getcwd() + '/deals', page), 'r').read()
    soup = BeautifulSoup(file, 'html.parser')
    
    ## seller names
    try:
        seller_names = soup.find('span', attrs = {'class':'mbg-nw'}).get_text()
    except:
        seller_names = "Null"
    
    
    
    ## seller score
    try:
        seller_scores = soup.find('span', attrs={'class': 'mbg-l'}).find('a').get_text()
    except:
        seller_scores = "Null"
        
        
        
    ## item price
    try:
        try:
            item_prices_US = soup.find('span', attrs={'id': 'prcIsum', 'class': 'notranslate'}).get_text()
            item_prices = pd.to_numeric(re.sub(r'.*\$([0-9]+).', r'\1', item_prices_US))
        except: 
            item_prices_US = soup.find('span', attrs={'id': 'mm-saleDscPrc', 'class': 'notranslate'}).get_text()
            item_prices = pd.to_numeric(re.sub(r'.*\$([0-9]+).', r'\1', item_prices_US))
    except:
        item_prices = "0"
        

        
    ## list price_yes
    try:
        list_prices_box = soup.find('span', attrs={'id': 'orgPrc', 'class': 'notranslate ms-orp'}).get_text()
        list_prices_yes = pd.to_numeric(re.sub(r'([A-Za-z]+..)(.*)', r'\2' , list_prices_box))
        if list_prices_yes > 0:
            list_prices_yes = 'Yes'
        else:
            list_prices_yes = 'No'
    except:
        list_prices_yes = "No"

        
    ## list price
    try:
        list_prices_box = soup.find('span', attrs={'id': 'orgPrc', 'class': 'notranslate ms-orp'}).get_text()
        list_prices_1 = pd.to_numeric(re.sub(r'([A-Za-z]+..)(.*)', r'\2' , list_prices_box))
        list_prices = list_prices_1 * 100
    except:
        list_prices = "0"
    
    
    
    ## number items sold
    try:
        sold = soup.find('a', class_ = 'vi-txt-underline').get_text()
        num_items_sold_1 = re.sub(r'([0-9]+) sold', r'\1', sold)
        num_items_sold = pd.to_numeric(re.sub(r',', r'', num_items_sold_1))

    except:
        num_items_sold = "0"
    
    
    
    ## title
    try:
        title_text =  soup.find('h1', attrs = {'class': 'it-ttl'}).get_text()
        title = re.sub(r'Details about  \xa0', r'', title_text)
    except:
        title_text = "Null"

        
        
    ## returns
    try:
        returns =  soup.find('span', attrs = {'id': 'vi-ret-accrd-txt'}).get_text()
        if returns == 'No returns':
            returns = 'No'
        else:
            returns = 'Yes'
    except:
        returns = "Null"

        
        
    ## shipping price
    try:
        shipping_price_info = soup.find('span', attrs={'id': 'fshippingCost', 'class': 'notranslate sh-fr-cst'}).get_text()
        shipping_price = re.sub(r'\n', r'', shipping_price_info)
        if shipping_price == 'FREE':
            shipping_price = '0'
        else:
            shipping_price = shipping_price
    except:
        shipping_price = shipping_price
        
    
    ## condition
    try:
        conditions = soup.find('div', attrs={'id': 'vi-itm-cond', 'class': 'u-flL condText'}).get_text()
    except:
        conditions = "Null"

    deals_df.loc[i] = [seller_names, seller_scores, item_prices, list_prices, list_prices_yes, num_items_sold, title, returns, shipping_price, conditions]
    i = i + 1

In [300]:
deals_df

,seller_names,seller_scores,item_prices,list_prices,list_prices_yes,num_items_sold,title,returns,shipping_price,conditions
0,Regency Rugs,65498,9499,40595,Yes,0,Large Traditional 8x11 Oriental Area Rug Persi...,Yes,0,New
1,luxdecorcollection,20396,2299,0,No,4017,6 Piece Bed Sheets Deep Pocket Egyptian Comfor...,Yes,0,New with tags
2,ecitydirect,99656,7899,0,No,570,Chezmoi Collection Seville 9-piece Jacquard Pa...,Yes,0,New with tags
3,abcotech,2031,0,0,No,188,Abco Tech Memory Foam Knee Pillow with Cooling...,Yes,0,New with tags
4,linenspa,22158,5299,0,No,1325,"Linenspa 2, 3, 4 inch Soft Plush Swirl Gel Mem...",Yes,0,New with tags
...,...,...,...,...,...,...,...,...,...,...
178,linenspa,22158,5299,0,No,2326,"LUCID 2, 3, 4 Inch Lavender Memory Foam Mattre...",Yes,0,New with tags
179,ienjoyhome,108983,1799,8999,Yes,4564,Egyptian Comfort Hotel Quality 4-Piece Bed She...,Yes,0,New with tags
180,ienjoyhome,108983,1899,8999,Yes,32,Home Collection Premium Ultra Soft Pattern 4 P...,Yes,0,New with tags
181,ecitydirect,99656,3699,0,No,1662,7pc Western Star Embroidery Microsuede Oversiz...,Yes,0,New with tags


# Save information above into SQL database

In [190]:
conn = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd="jiayou1993929"
)

cursor = conn.cursor()
cursor.execute("USE msba")
cursor.execute("CREATE TABLE deals (seller_names VARCHAR(255),seller_scores INT,item_prices INT,list_prices INT,num_items_sold VARCHAR(255),title VARCHAR(255),returns VARCHAR(255),shipping_price VARCHAR(255), conditions VARCHAR(255))")

In [199]:
for j in deals_df.index:
    sql = "INSERT INTO deals (seller_names, seller_scores, item_prices, list_prices, num_items_sold, title, returns, shipping_price, conditions) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    value = (deals_df['seller_names'][j], deals_df['seller_scores'][j],  deals_df['item_prices'][j], deals_df['list_prices'][j], deals_df['num_items_sold'][j],  deals_df['title'][j], deals_df['returns'][j], deals_df['shipping_price'][j], deals_df['conditions'][j])
    cursor.execute(sql, value)
    conn.commit()

# Run summary statistics on each item

Print the mean, min, max, and mean for each numeric column, dependent on whether “list price” and "condition" was provided

In [303]:
deals_df['seller_scores'] = pd.to_numeric(deals_df['seller_scores'])
deals_df['item_prices'] = pd.to_numeric(deals_df['item_prices'])
deals_df['list_prices'] = pd.to_numeric(deals_df['list_prices'])
deals_df['num_items_sold'] = pd.to_numeric(deals_df['num_items_sold'])
deals_df['shipping_price'] = pd.to_numeric(deals_df['shipping_price'])

In [306]:
# Build a table
tables = deals_df[['list_prices_yes', 'conditions', 'seller_scores']].groupby(['list_prices_yes', 'conditions'], as_index = False).mean()
tables = tables.drop('seller_scores',1)
tables

In [307]:
# Mean
mean_ss = deals_df[['list_prices_yes', 'conditions', 'seller_scores']].groupby(['list_prices_yes', 'conditions'], as_index = False).mean()
mean_it = deals_df[['list_prices_yes', 'conditions', 'item_prices']].groupby(['list_prices_yes', 'conditions'], as_index = False).mean()
mean_sp = deals_df[['list_prices_yes', 'conditions', 'shipping_price']].groupby(['list_prices_yes', 'conditions'], as_index = False).mean()
mean_nis = deals_df[['list_prices_yes', 'conditions', 'num_items_sold']].groupby(['list_prices_yes', 'conditions'], as_index = False).mean()
mean_summary = pd.merge(pd.merge(pd.merge(mean_ss, mean_it, on = ['list_prices_yes', 'conditions']), mean_sp, on = ['list_prices_yes', 'conditions']), mean_nis, on = ['list_prices_yes', 'conditions'])
mean_summary.rename(columns = {'seller_scores':'mean_seller_scores', 'item_prices':'mean_item_prices', 'shipping_price':'mean_shipping_price', 'num_items_sold':'mean_num_items_sold'}, inplace = True)


# Max
max_ss = deals_df[['list_prices_yes', 'conditions', 'seller_scores']].groupby(['list_prices_yes', 'conditions'], as_index = False).max()
max_it = deals_df[['list_prices_yes', 'conditions', 'item_prices']].groupby(['list_prices_yes', 'conditions'], as_index = False).max()
max_sp = deals_df[['list_prices_yes', 'conditions', 'shipping_price']].groupby(['list_prices_yes', 'conditions'], as_index = False).max()
max_nis = deals_df[['list_prices_yes', 'conditions', 'num_items_sold']].groupby(['list_prices_yes', 'conditions'], as_index = False).max()
max_summary = pd.merge(pd.merge(pd.merge(max_ss, max_it, on = ['list_prices_yes', 'conditions']), max_sp, on = ['list_prices_yes', 'conditions']), max_nis, on = ['list_prices_yes', 'conditions'])
max_summary.rename(columns = {'seller_scores':'max_seller_scores', 'item_prices':'max_item_prices', 'shipping_price':'max_shipping_price', 'num_items_sold':'max_num_items_sold'}, inplace = True)


# Min
min_ss = deals_df[['list_prices_yes', 'conditions', 'seller_scores']].groupby(['list_prices_yes', 'conditions'], as_index = False).min()
min_it = deals_df[['list_prices_yes', 'conditions', 'item_prices']].groupby(['list_prices_yes', 'conditions'], as_index = False).min()
min_sp = deals_df[['list_prices_yes', 'conditions', 'shipping_price']].groupby(['list_prices_yes', 'conditions'], as_index = False).min()
min_nis = deals_df[['list_prices_yes', 'conditions', 'num_items_sold']].groupby(['list_prices_yes', 'conditions'], as_index = False).min()
min_summary = pd.merge(pd.merge(pd.merge(min_ss, min_it, on = ['list_prices_yes', 'conditions']), min_sp, on = ['list_prices_yes', 'conditions']), min_nis, on = ['list_prices_yes', 'conditions'])
min_summary.rename(columns = {'seller_scores':'min_seller_scores', 'item_prices':'min_item_prices', 'shipping_price':'min_shipping_price', 'num_items_sold':'min_num_items_sold'}, inplace = True)


,list_prices_yes,conditions,mean_seller_scores,mean_item_prices,mean_shipping_price,mean_num_items_sold
0,No,New,59763.800000,1702.400000,0,1622.700000
1,No,New with tags,77527.750000,2197.150000,0,1294.080000
2,Yes,New,73284.000000,2337.888889,0,4143.333333
3,Yes,New with tags,192600.328125,2274.921875,0,8342.562500


In [312]:
summary = pd.merge(pd.merge(mean_summary, min_summary, on = ['list_prices_yes', 'conditions']), max_summary, on = ['list_prices_yes', 'conditions'])
summary

,list_prices_yes,conditions,mean_seller_scores,mean_item_prices,mean_shipping_price,mean_num_items_sold,min_seller_scores,min_item_prices,min_shipping_price,min_num_items_sold,max_seller_scores,max_item_prices,max_shipping_price,max_num_items_sold
0,No,New,59763.800000,1702.400000,0,1622.700000,20535,550,0,59,146977,3999,0,6267
1,No,New with tags,77527.750000,2197.150000,0,1294.080000,177,0,0,0,799164,7980,0,6979
2,Yes,New,73284.000000,2337.888889,0,4143.333333,65498,999,0,0,77177,9499,0,28135
3,Yes,New with tags,192600.328125,2274.921875,0,8342.562500,3196,599,0,0,712377,7699,0,181435


# Conclusion

From the table above, it seems that the existence of “list price” appear to influence sales
The items with list prices have higher mean number of items sold regardless of the conditions